In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

In [85]:
url = 'https://github.com/jwkuyper/IOWA_test/blob/main/Dataset_V1.xlsx?raw=true'
df = pd.read_excel(url,sheet_name='Sheet1')


null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

In [86]:
# Filling all the null/NaN values of new* with 0
def FillNullWithZero(Dataframe, ColName):
    if ColName.startswith("new") or ColName.startswith("total") or "smokers" in ColName:
        Dataframe = Dataframe.fillna(value = {ColName:0})
    else:
        pass
    return Dataframe

for i in range(len(df.columns)):
    df = FillNullWithZero(df, df.columns[i])


In [87]:
#Dropping the rows which does not have any continent 
#Comment to be deleted: Dropping these 586 entries as these are identified as World and International, also most of their other features are all null
df.drop(df[df.continent.isnull()].index, inplace=True)

#comment TBD: My recommendation is to ddrop positive_rate column too.
df = df.drop("positive_rate", axis=1)

df = df.fillna(value={"stringency_index": df.stringency_index.mean(),
                      "population_density": df.population_density.min(),
                      "median_age": df.median_age.mean(),
                      "gdp_per_capita":df.gdp_per_capita.mean(),
                      "hospital_beds_per_thousand":df.hospital_beds_per_thousand.mean(),
                      "life_expectancy": df.life_expectancy.mean()
                     }
              )


In [88]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Series([], dtype: float64)

In [89]:
print(df[df.isnull().any(axis=1)][null_columns].head(10))

Empty DataFrame
Columns: []
Index: []


In [90]:
df.iso_code = pd.Categorical(df.iso_code)

In [91]:
df.iso_code = df.iso_code.cat.codes

In [92]:
df.sample(n=10, replace=False).to_csv(path_or_buf= 'sample.csv', index=False)

In [93]:
df.drop(['continent', 'location', 'date'], axis=1,inplace=True)

In [94]:
features = list(df.columns)
features.remove('new_cases')
features

['iso_code',
 'total_cases',
 'total_deaths',
 'new_deaths',
 'total_cases_per_million',
 'new_cases_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'stringency_index',
 'population',
 'population_density',
 'median_age',
 'gdp_per_capita',
 'female_smokers',
 'male_smokers',
 'hospital_beds_per_thousand',
 'life_expectancy']

In [95]:
target = "new_cases"

In [96]:
X =df[features]
y = df[target]

In [97]:
from sklearn.model_selection import train_test_split      # train/test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [98]:
from sklearn.linear_model import LinearRegression #linear regression
lr=LinearRegression()

In [99]:
lr.fit(X_train, y_train)

LinearRegression()

In [100]:
lr.score(X_train,y_train)

0.8789884207297399

In [101]:
print(lr.score(X_test,y_test))

0.8866321603945762


In [102]:
from sklearn.neighbors import KNeighborsClassifier

In [103]:
knn = KNeighborsClassifier(n_neighbors=5)

In [104]:
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [105]:
knn.score(X_train,y_train)

0.44881097399502146

In [106]:
knn.score(X_test,y_test)

0.32801080394026055

With the below sample data the new_cases (output) should be 246

In [108]:
d = {"iso_code": 103,
"total_cases": 32364,
"total_deaths": 548,
"new_deaths": 6,
"total_cases_per_million": 601.882,
"new_cases_per_million": 4.575,
"total_deaths_per_million": 10.191,
"new_deaths_per_million": 0.112,
"new_tests": 3381,
"total_tests": 425364,
"total_tests_per_thousand": 7.911,
"new_tests_per_thousand": 0.063,
"stringency_index": 70.37,
"population": 53771300,
"population_density": 87.324,
"median_age": 20,
"gdp_per_capita": 2993.028,
"female_smokers": 1.2,
"male_smokers": 20.4,
"hospital_beds_per_thousand": 1.4,
"life_expectancy": 66.7}

new_case = [d["iso_code"], d["total_cases"], d["total_deaths"], d["new_deaths"],
            d["total_cases_per_million"], d["new_cases_per_million"], d["total_deaths_per_million"],
            d["new_deaths_per_million"], d["new_tests"], d["total_tests"], d["total_tests_per_thousand"],
            d["new_tests_per_thousand"], d["stringency_index"], d["population"], d["population_density"],
            d["median_age"], d["gdp_per_capita"], d["female_smokers"], d["male_smokers"], d["hospital_beds_per_thousand"],
            d["life_expectancy"]]
X_new = [new_case]
knn.predict(X_new)

array([213.])